In [ ]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
import os
from forex_python.converter import CurrencyRates
from datetime import datetime

In [ ]:
print("Welcome to the Financial Instrument Visual Evaluator.")
print("You’ll be prompted to enter tickers, instrument types, sectors, and other data.")
print("We'll collect price and fundamental data and generate charts for evaluation.")

In [ ]:
# Data Input Method Selection
input_method = input("Select input method (1 for CSV, 2 for manual entry): ").strip()

df = pd.DataFrame(columns=["Ticker", "Instrument Type", "Sector", "Region", 
                          "Allocation (%)", "Currency", "Conversion Rate (to TWD)"])

# Option 1: Load from CSV
if input_method == "1":
    csv_file = input("Enter CSV filename (default: Instrument_Allocation_Template.csv): ").strip()
    if not csv_file:
        csv_file = "Instrument_Allocation_Template.csv"
    
    # Check if the CSV file exists
    if not os.path.isfile(csv_file):
        print(f"Error: The file '{csv_file}' does not exist.")
        print("Please create this file with the following columns:")
        print("Ticker, Instrument Type, Sector, Region, Allocation (%), Currency, Conversion Rate (to TWD)")
        print("Example:")
        print("AAPL,Stock,Technology,US,20,USD,")
        print("2330.TW,Stock,Technology,Taiwan,30,TWD,")
    else:
        try:
            # Read the CSV file
            df = pd.read_csv(csv_file)
            print(f"Successfully loaded {len(df)} instruments from {csv_file}")
            
            # Validate required columns
            required_columns = ["Ticker", "Instrument Type", "Sector", "Region", 
                               "Allocation (%)", "Currency"]
            missing_columns = [col for col in required_columns if col not in df.columns]
            
            if missing_columns:
                print(f"Warning: The following required columns are missing: {missing_columns}")
                print("Please update your CSV file with the correct column headers.")
        except Exception as e:
            print(f"Error loading CSV file: {e}")

# Option 2: Manual Entry
elif input_method == "2":
    instruments = []
    print("\nEnter information for each instrument:")
    print("(Enter 'done' for ticker when finished)")
    
    while True:
        ticker = input("\nEnter ticker symbol (or type 'done' to finish): ").strip()
        if ticker.lower() == 'done':
            break
        instrument_type = input("Instrument Type (Stock, ETF, Bond, etc.): ").strip()
        sector = input("Sector (e.g., Technology, Healthcare): ").strip()
        region = input("Region (US, Europe, Asia-Pacific, etc.): ").strip()
        
        while True:
            try:
                allocation = float(input("Allocation percentage (0-100): ").strip())
                break
            except ValueError:
                print("Please enter a valid number.")
        
        currency = input("Currency (USD or TWD): ").strip().upper()

        instruments.append({
            "Ticker": ticker,
            "Instrument Type": instrument_type,
            "Sector": sector,
            "Region": region,
            "Allocation (%)": allocation,
            "Currency": currency,
            "Conversion Rate (to TWD)": None
        })

    df = pd.DataFrame(instruments)
    print(f"Successfully added {len(df)} instruments manually.")

else:
    print("Invalid selection. Please run the program again and select 1 or 2.")

In [ ]:
c = CurrencyRates()
try:
    usd_to_twd = c.get_rate('USD', 'TWD')
    df.loc[df['Currency'] == 'USD', 'Conversion Rate (to TWD)'] = usd_to_twd
    df.loc[df['Currency'] == 'TWD', 'Conversion Rate (to TWD)'] = 1.0
    print(f"USD to TWD rate: {usd_to_twd}")
except Exception as e:
    print(f"Currency conversion failed: {e}")
    df['Conversion Rate (to TWD)'] = 1.0  # fallback

In [ ]:
price_data = {}
for ticker in df['Ticker']:
    try:
        stock = yf.Ticker(ticker)
        hist_daily = stock.history(period="1y", interval="1d")
        hist_weekly = stock.history(period="5y", interval="1wk")
        price_data[ticker] = {"1y_daily": hist_daily, "5y_weekly": hist_weekly}
        print(f"Downloaded data for {ticker}")
    except Exception as e:
        print(f"Failed to fetch data for {ticker}: {e}")

In [ ]:
# Plot price + moving averages for all tickers
for ticker in df['Ticker']:
    if ticker not in price_data:
        print(f"Skipping {ticker} due to missing data.")
        continue
    
    data = price_data[ticker]['1y_daily']
    data['MA50'] = data['Close'].rolling(window=50).mean()
    data['MA200'] = data['Close'].rolling(window=200).mean()
    
    plt.figure(figsize=(12, 6))
    plt.plot(data['Close'], label='Close Price')
    plt.plot(data['MA50'], label='50-day MA')
    plt.plot(data['MA200'], label='200-day MA')
    plt.title(f"{ticker} - 1Y Daily Price with Moving Averages")
    plt.xlabel("Date")
    plt.ylabel("Price (USD)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()